In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fasttext konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 38.4 MB/s eta 0:00:00
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313503 sha256=be9ccd5c2219019cdac7001e5f5fcaf3eceae6dce97aa3907178a0ede16488e4
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
# ▒▒ 0) 환경 설정 ▒▒──────────────────────────────────────────
import os, json, re, pickle, random
from pathlib import Path
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import fasttext
from konlpy.tag import Okt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from tqdm.auto import tqdm

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# 작업 경로들 (필요하면 수정)
DATA_PATH   = Path("/content/drive/MyDrive/25-1-Bridge/combined_labeled_reviews.json")   # ← 새 JSON
FT_MODEL    = Path("/content/drive/MyDrive/25-1-Bridge/cc.ko.300.bin")
SAVE_DIR    = Path("/content/drive/MyDrive/25-1-Bridge/OUTPUT"); SAVE_DIR.mkdir(exist_ok=True)

# 학습 하이퍼파라미터
BATCH_SIZE      = 128
PAD_LEN         = 800      # 문장 당 단어 벡터 수
EMB_DIM         = 300
HIDDEN_DIM      = 32
EPOCHS          = 1000
PATIENCE        = 100      # early lr-decay
LR              = 1e-4

# ▒▒ 1) 데이터 로드 ▒▒────────────────────────────────────────
def load_json(path: Path):
    """새 포맷: [{"review": "...", "label": "ad"|"non_ad"}, ...]"""
    with open(path, "r", encoding="utf-8") as fp:
        data = json.load(fp)
    df = pd.DataFrame(data)
    # label 매핑: ad →1, non_ad →0
    df["label"] = df["label"].map({"ad": 1, "non_ad": 0})
    return df

df = load_json(DATA_PATH)
print("총 샘플:", len(df))

# ▒▒ 2) 불용어 사전 구축 ▒▒───────────────────────────────────
# 2-1) 사용자 수동 리스트
STOPWORDS = ['이', '그', '저', '은', '는', '이었', '합니다', '그리고', '하지만', '을', '를', '에', '의', '가',
             '~', ':)', '!', ')', '(', '.', ',', '쪽지', '공감', '스크랩', '익명', '(?)', '?', '!', '~~',
             '후기', '연세대', '서문', '신고', '・', 'URL', '복사', '선택', '추천', '네이버', '리뷰', '식당',
             '음식', '랑', '와', '으로', '으로는', '-', '이랑', '야', '옆', '까지', '께서', '도', '앞', '지만',
             '그런데', '사실', '이라', '내', "'", '다만', ':', '..', '보다', '너무', '더', '진짜', '개', '과',
             '좀', '들', '한', '님', '하고', '든', '전', '계속', '인데', '이라는', '법', '+', '쪽', '음', '아',
             '정도', '원', '맵', '못', '이었고', '꽤', '총', '평', '이고', '분', '세', '조금', '보니', '으로서',
             '아래', '쯤', '본문', '추가', '이웃', '여기', '주위', '극', '대', '노', '대박', '모', '것', '밖에',
             '그냥', '에서도', '할', '근대', '같이', '처럼', '신촌', '고서', '평소', '네', '번째', '별로', '수도',
             '우리', '로', '안', '글', '거', '나', '집', '다', 'ㅎㅎ', 'ㅎ', 'ㅋ', '짜리', '기타', '돼서', '느낌',
             '적', '본', 'ㅋㅋ', '방문', '봐서', '상황', '한번', '에서', '해요', '요', '오', '내돈내산', '솔직후기',
             '제공', '광고성', '광고', '2024', '2023', '2022', '시간', '분', '전', 'AM', 'PM', 'am', 'pm',
             '이용', '위치', '맛집', '시간', '주문', '메뉴', '예약', '영업',
             '맛', '근처', '소개', '사진', '위해', '여기', '에서는', '에서',
             '곳', '다녀왔어요', '먹었어요', '갔습니다', '합니다', '입니다',
             '했습니다', '다음', '누구', '특히', '때', '있는', '매일',
             '가능', '추천', '검색', '잘', '다', '된', '중', '와서',
             '맛있는', '수', '곳', '있는', '해서', '직접', '새로',
             '하고', '하시', '저녁', '점심', '아침',
             '가능한', '정말', '많은', '모두', '보다는',  '하는',
             '해주시는', '휴무', '매주', '필요', '좋아요', '많아서', '다녀오고',
             '게', '는', '을', '를', '이', '가', '의', '에', '와', '과',
             '로', '으로', '도', '다', '만', '보다', '부터', '그리고',
             '또는', '해서', '인데', '이라', '이며', '또', '그', '하지만',
             '그래도', '때문에', '이런', '저런', '그런', '더', '까지',
             '하면', '든', '요', '하니', '하면서', '으로써', '중에', '처럼','쓰고있다고','때문',
             '갈','있어요','이었는데','에는','마다','없이','했어요','있었어요','랬','적고',
             '있었기','했는지','참','있고','있도록','되어','라고','좋았요','니당','어떤','리기',
             '있었던','리도','잇','하길래','있다','일까','지','되는','아닌','고','D','등','통해',
             '데','겸','않고','S','이나','무엇','또한','드립니다','A','하겠습니다','보면','그렇다면',
             '있어서','학','라','집니다','면','에는','T','X','니','하는데요','에게','이었네요','듯',
             '있','o','set','같아용','자아내죠','건','Instagram','대해','이건','이었습니다',
             'Rachadamnoen','보이는','각종','이서','였다','있었습니다','싶은','되어있어서',
             '하십니다','였어요','ㅋㅋㅋ','채','OO','ㅡ','그렇게','이서','건가','앗','였어요','저희',
             '했을','듬뿍','넘','단체','같네요','메뉴','국물', '도삭면', '백김치', '떡갈비', '팟타이',
             '옹심이', '게장', '계란말이', '어묵', '반찬', '만두', '한우', '육전','튀김', '초밥', '국물', '닭발', '목살', '치킨',
             '꼬치', '비빔국수', '들깨', '김밥', '해장국', '반찬', '쌈무', '돈까스','닭발', '치킨', '들깨', '장탕', '새우',
             '국수', '카츠','김밥', '갈비', '김치', '냉면','인가요','됐습니다','되었더라구요','Mueang','그만','치고는',
             '있냐면','인','되어있었다','팅','씩',' ','버거','찐','이렇게','쏙','제','이네요','및','해도','어느','로서','ㄹ','놀',
             '뭔','하지','잇다고','였던','따라','합니당','부탁드립니당','이제','끔','꼭','에도','한다는','하','있습니다','스럽게',
             '드림다','하신다면','엔','에도','꼭','최고','좋은','같아요','아마','ㅠㅠ','바로','','오늘','딱','함께','이지만',
             '외','씩','ERang','naver','뿐','NO','nail','go','ㅎㅎㅎ','위','인','껏','뜨','속','봅니다','크','더욱','려고',
             '엄청','좋은','했던','있게','래서','한데','있는데','내내','예요','이니까','이다','역시','굿','층','우삼겹',
             '움','덕분','그럼에도','g','이었다','일단','요즘','계란찜','가장','짠','하다','이라고','했다','즈','아주',
             '자주','모든','되었습니다','공유','댓글','혹시','갈치','구이','ㅠ','샤브샤브','소반','이에요','middot','라는',
             '움','닭갈비','쌈','거야','익히','같았어요','라곤','한테','걸','이었던','서','3','걸','했는데요','싶네요','까요',
             '파스타','있다면','있다고','익히','라곤','이었던','라면','멍멍','으로도','했었던','그래서','고고','동안',
             'o','꿔','천','아니라','매우','다녀오실수있어용','에까지','서','니까','인지','있으며','죠','K','라면','물론',
             'OGQ','me','thumb','나야','겠죠','두','난','에게는','젤루','K','하구요','면서','하는데','될','하기에','껄',
             'ㅋㅋㅋㅋㅋ','뭐','d','you','Chang','가츠덮밥','쿠시카츠','미역국','쭈구미','갈비탕','평일','부타동','메론빵',
             '껍항정','그리','거기','mp','막국수','돈까스','치즈스틱','모듬전','ㅋㅋㅋㅋ','ᵔ','같은','에요','여','각','라멘',
             '다른','보통','갑니당','곱창','연어','이었어요','수육','김치찌개','한다면','하루하루','ㅠㅠㅠ','    ','장어','돼지국밥',
             '대로','으','완전','오후','무렵','넉','구','에서만','시','햄버거','피자','쭈꾸미', "호텔", "객실", "교통", "시설",
             "오송", "강남", "남원", "서귀포시", "장소", "터미널", "수쿰빗", "하카타역", "마카오", "디럭스", "르", "홍대"
             ]

# 2-2) 모든 날짜·시각 문자열을 불용어에 추가
def gen_dates(start=2022, end=2024):
    d0, d1 = datetime(start,1,1), datetime(end,12,31)
    return [(d0+timedelta(days=i)).strftime("%Y.%-m.%-d.")  # 2024.5.18. 형태
            for i in range((d1-d0).days+1)]

def gen_times():
    return [f"{h:02}:{m:02}" for h in range(24) for m in range(60)]

STOPWORDS += gen_dates() + gen_times()

# 저장(선택)
with open(SAVE_DIR/"stopwords.pkl", "wb") as fp:
    pickle.dump(STOPWORDS, fp)

# ▒▒ 3) FastText 모델 & 전처리 함수 ▒▒───────────────────────
okt = Okt()
ft  = fasttext.load_model(str(FT_MODEL))

HASHTAG_RE = re.compile(r"#\S+")

def preprocess(text: str):
    """해시태그 제거+형태소 분리+불용어 제거+한글•영어 알파벳만 남김"""
    text = HASHTAG_RE.sub("", str(text)).strip()
    toks = okt.morphs(text)
    return [t for t in toks if t.isalpha() and t not in STOPWORDS]

def sent2matrix(tokens):
    """토큰 리스트 → (PAD_LEN, EMB_DIM) 행렬"""
    vecs = [ft.get_word_vector(tok) for tok in tokens]
    if len(vecs) < PAD_LEN:
        vecs += [np.zeros(EMB_DIM)] * (PAD_LEN - len(vecs))
    else:
        vecs = vecs[:PAD_LEN]
    return np.array(vecs, dtype=np.float32)

# 전체 문장 임베딩 캐싱 (→ 메모리 충분할 때)
token_lists = []
matrices = []
for txt in tqdm(df["review"], desc="Embedding"):
    toks = preprocess(txt)
    token_lists.append(toks)
    matrices.append(sent2matrix(toks))

X = np.stack(matrices)               # (N, PAD_LEN, EMB_DIM)
y = df["label"].values.astype(np.float32)

np.save(SAVE_DIR/"embeddings.npy", X)

# ▒▒ 4) Dataset & DataLoader ▒▒──────────────────────────────
class ReviewDS(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x)
        self.y = torch.tensor(y)
    def __len__(self):  return len(self.y)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

X_tr, X_te, y_tr, y_te, toks_tr, toks_te = train_test_split(
    X, y, token_lists, test_size=0.2, stratify=y, random_state=SEED)

tr_dl = DataLoader(ReviewDS(X_tr, y_tr), batch_size=BATCH_SIZE, shuffle=True)
te_dl = DataLoader(ReviewDS(X_te, y_te), batch_size=1)

# ▒▒ 5) LSTM+Self-Attention 모델 ▒▒──────────────────────────
class LSTMAttn(nn.Module):
    """한 층 LSTM + 간단한 Self-Attention(weight sharing) + 이진 분류"""
    def __init__(self, in_dim=EMB_DIM, hid=HIDDEN_DIM, seq_len=PAD_LEN):
        super().__init__()
        self.lstm   = nn.LSTM(in_dim, hid, batch_first=True)
        self.w_attn = nn.Linear(seq_len, seq_len, bias=False)  # (B,h,seq)→(B,h,seq)
        self.fc     = nn.Linear(hid*seq_len, 1)
        self.flat   = nn.Flatten()

    def forward(self, x):                     # x: (B,seq,in_dim)
        h, _   = self.lstm(x)                 # (B,seq,hid)
        p      = h.permute(0, 2, 1)           # (B,hid,seq)
        q      = self.w_attn(p)               # (B,hid,seq)
        attn   = h * q.permute(0, 2, 1)       # (B,seq,hid) element-wise
        logit  = self.fc(self.flat(attn))     # (B,1)
        return logit.squeeze(1), attn         # (B), (B,seq,hid)

model     = LSTMAttn().to("cuda" if torch.cuda.is_available() else "cpu")
criterion  = nn.BCEWithLogitsLoss()
optimizer  = torch.optim.Adam(model.parameters(), lr=LR)

def binarize(out):   # 로짓 → 0/1 tensor
    return (torch.sigmoid(out) > .5).int()


총 샘플: 2080


Embedding:   0%|          | 0/2080 [00:00<?, ?it/s]

In [ ]:
# ▒▒ 6) 학습 루프  (LR Decay + Early Stopping) ▒▒──────────────────────
BATCH_SIZE = 128        # (이미 선언돼 있으면 생략해도 무방)

LR_PATIENCE  = 100      # validation loss가 개선되지 않을 때 LR 감소까지 기다릴 에포크 수
ES_PATIENCE  = 150      # validation loss가 개선되지 않을 때 조기 종료까지 기다릴 에포크 수
LR_FACTOR    = 0.1      # LR을 몇 배로 줄일지
MIN_LR       = 1e-7     # 더 이상 줄이지 않을 최소 LR
IMPROVE_DELTA = 1e-4    # loss 개선으로 인정할 최소 변화량

best_loss          = float("inf")
lr_wait, es_wait   = 0, 0            # 두 개의 카운터
device             = next(model.parameters()).device

for epoch in range(1, EPOCHS + 1):
    # ─────────── Train ───────────
    model.train()
    train_loss, correct = 0.0, 0
    for xb, yb in tr_dl:
        xb, yb = xb.to(device), yb.to(device)
        logit, _ = model(xb)
        loss = criterion(logit, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * len(yb)
        correct    += (binarize(logit) == yb.int()).sum().item()

    train_loss /= len(y_tr)
    train_acc   = correct / len(y_tr)

    # ─────────── Validation ───────
    model.eval()
    val_loss, correct = 0.0, 0
    with torch.no_grad():
        for xb, yb in te_dl:
            xb, yb = xb.to(device), yb.to(device)
            logit, _ = model(xb)
            loss = criterion(logit, yb)

            val_loss += loss.item()
            correct  += (binarize(logit) == yb.int()).sum().item()

    val_loss /= len(y_te)
    val_acc   = correct / len(y_te)

    # ─────────── 로그 출력 ────────
    if epoch % 50 == 0 or epoch == 1:
        print(f"[{epoch:4d}/{EPOCHS}]  "
              f"train:{train_loss:.4f}/{train_acc:.3f}  "
              f"val:{val_loss:.4f}/{val_acc:.3f}")

    # ─────────── 개선 여부 판단 ────
    if best_loss - val_loss > IMPROVE_DELTA:   # 충분히 개선
        best_loss = val_loss
        lr_wait   = 0
        es_wait   = 0
        torch.save(model.state_dict(), SAVE_DIR / "best_model.pth")
    else:
        lr_wait += 1
        es_wait += 1

    # ─────────── LR decay ──────────
    if lr_wait >= LR_PATIENCE:
        for group in optimizer.param_groups:
            new_lr = max(group["lr"] * LR_FACTOR, MIN_LR)
            group["lr"] = new_lr
        lr_wait = 0
        print(f"→ LR decay, now {optimizer.param_groups[0]['lr']:.1e}")

    # ─────────── Early Stopping ────
    if es_wait >= ES_PATIENCE:
        print(f"\n◆ Early Stopping at epoch {epoch} (no val-loss improvement for {ES_PATIENCE} epochs)")
        break

print("Best val loss:", best_loss)

# ▒▒ 7) 전체 데이터 성능 & F1 ▒▒──────────────────────────────
model.load_state_dict(torch.load(SAVE_DIR / "best_model.pth"))
model.eval()
with torch.no_grad():
    logits, attn = model(torch.tensor(X).to(device))
pred = binarize(logits).cpu().numpy()
f1   = f1_score(y, pred)
acc  = accuracy_score(y, pred)
print(f"\n전체 데이터 F1 = {f1:.4f}  ACC = {acc:.4f}")

# ▒▒ 9) 마무리 저장 ▒▒────────────────────────────────────────
torch.save(model.state_dict(), SAVE_DIR / "model_weights.pth")
print("\n모델 가중치를", SAVE_DIR / "model_weights.pth", "에 저장 완료!")


[   1/1000]  train:0.6878/0.797  val:0.6792/0.505
[  50/1000]  train:0.0181/0.995  val:0.0252/0.995
[ 100/1000]  train:0.0003/1.000  val:0.0422/0.995
[ 150/1000]  train:0.0001/1.000  val:0.0538/0.995
→ LR decay, now 1.0e-05
[ 200/1000]  train:0.0000/1.000  val:0.0547/0.995

◆ Early Stopping at epoch 201 (no val-loss improvement for 150 epochs)
Best val loss: 0.02492677217445064

전체 데이터 F1 = 0.9957  ACC = 0.9957

모델 가중치를 /content/drive/MyDrive/25-1-Bridge/OUTPUT/model_weights.pth 에 저장 완료!


In [ ]:
# ▒▒ 8) 어텐션이 집중된 단어 확인 ▒▒────────────────────────
def top_tokens(att_mat, tokens, topk=5):
    """
    att_mat : (seq_len, hidden) – 한 문장서 각 토큰의 attention(가중치) 행렬
    tokens  : 해당 문장의 토큰 리스트
    """
    scores = att_mat.sum(dim=1)                 # (seq_len,)
    idxs   = torch.argsort(scores, descending=True)[:topk].tolist()
    return [tokens[i] for i in idxs if i < len(tokens)]

print("\n📝 리뷰별 상위 Attention 단어")
for i in range(20):   # 앞 5문장만 예시
    words = top_tokens(attn[i], token_lists[i])
    label = "광고" if y[i] else "일반"
    print(f"{i:02d} ({label}) → {words}")


📝 리뷰별 상위 Attention 단어
00 (광고) → ['정보', '장단점', '아고다', 'm', '수수료']
01 (광고) → ['KTX', '약', '이동만', '워크샵', '내부']
02 (광고) → ['둥지', '나이스', '원룸', '않는', '방']
03 (광고) → ['제주', '제주', '조식', '제주', '먹고']
04 (광고) → ['마켓', '쉐라톤', '투숙', '숙박', '날']
05 (광고) → ['불편함은', '체험', '홍보', '여행', '굳이']
06 (광고) → ['생활', 'LG', '제외', '반입', '음료']
07 (광고) → ['천안', '갑자기', '쉴', '절정', '앱']
08 (광고) → ['욕조', '어', '오히려', '목', '커서']
09 (광고) → ['않는', '인근', '점', '건물', '주중']
10 (광고) → ['센', '드린듯', '없으니', '차에', '여기는']
11 (광고) → ['익산', '새로운', '지원', '비', '행복']
12 (광고) → ['별도', '비발디', 'Ganglio', '홍천', '주방']
13 (광고) → ['링크', '적용', '코드', '스테이', '목욕']
14 (광고) → ['게임', '관광', '오픈', '수영', '가는']
15 (광고) → ['체크', '뷰', '림', '뷰', '트윈']
16 (광고) → ['라운지', '포함', '따로', '서울', '건물']
17 (광고) → ['에이치', '핫', '플', '전주', '바']
18 (광고) → ['건지', '대용', '주차', '담요', '그러세요']
19 (광고) → ['현대', '결제', '해야', '선', '사람']


In [ ]:
# ─────────────────────────────────────────────────────────────
# 셀 1: 저장된 최적 가중치 로드 후 validation(test) 성능 확인
# ─────────────────────────────────────────────────────────────
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data import DataLoader

VAL_BATCH = 32                           # 필요하면 조정
weights_path = SAVE_DIR / "best_model.pth"

# 1) 모델 초기화 & 가중치 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model_val = LSTMAttn().to(device)
model_val.load_state_dict(torch.load(weights_path, map_location=device))
model_val.eval()

# 2) Validation DataLoader
val_dl = DataLoader(ReviewDS(X_te, y_te), batch_size=VAL_BATCH)

# 3) 예측 & 지표 계산
all_preds, all_trues = [], []
with torch.no_grad():
    for xb, yb in val_dl:
        xb = xb.to(device)
        logits, _ = model_val(xb)
        preds = (torch.sigmoid(logits) > 0.5).cpu().int()
        all_preds.append(preds)
        all_trues.append(yb.int())
all_preds = torch.cat(all_preds).numpy()
all_trues = torch.cat(all_trues).numpy()

f1  = f1_score(all_trues, all_preds)
acc = accuracy_score(all_trues, all_preds)
print(f"[Validation] F1-score = {f1:.4f} | Accuracy = {acc:.4f}")


[Validation] F1-score = 0.9952 | Accuracy = 0.9952


In [ ]:
# ─────────────────────────────────────────────────────────────
# 셀 2: 전체 리뷰에 대해 광고일 확률(prob_ad) 계산하여 JSON에 추가
# ─────────────────────────────────────────────────────────────
FULL_BATCH = 256                        # GPU 메모리에 맞춰 조정

# 1) DataLoader 구성
full_dl = DataLoader(ReviewDS(X, y), batch_size=FULL_BATCH)

# 2) 확률 예측
probs = []
model_val.eval()
with torch.no_grad():
    for xb, _ in full_dl:
        xb = xb.to(device)
        logits, _ = model_val(xb)
        p = torch.sigmoid(logits).cpu().numpy()    # (B,)
        probs.extend(p.tolist())

# 3) 원본 DataFrame에 확률 열 추가 후 확인
df_with_prob = df.copy()               # 기존 df: review / label
df_with_prob["prob_ad"] = probs        # ad(1)일 확률

print(df_with_prob.head())

# (선택) JSON으로 저장하려면:
output_json_path = SAVE_DIR / "reviews_with_prob.json"
df_with_prob.to_json(output_json_path, orient="records", force_ascii=False, indent=2)
print(f"\n✓ 확률 추가 JSON 저장: {output_json_path}")


                                              review  label  prob_ad
0  더 좋은 여행 정보를 제공하기 위해 \n아고다와 함께 합니다.\n본 콘텐츠를 통한 ...      1      1.0
1  며칠 전 여수로 IP 워크샵을 다녀왔다.\n당일 오후 2시 워크샵 시작이라 오전 시...      1      1.0
2  주의\n이 글은 더이상 작성하지 않는 글입니다. 차후 삭제 될수 있습니다.\n[이전...      1      1.0
3  제주 웰니스 WE호텔\n다양한 힐링 프로그램이 있는 웰니스 WE호텔\nWE호텔\n제...      1      1.0
4  방콕여행 3일차 : 페닌슐라 방콕 조식뷔페 - 방콕 쉐라톤 그랜드 수쿰빗 숙박 후기...      1      1.0

✓ 확률 추가 JSON 저장: /content/drive/MyDrive/25-1-Bridge/OUTPUT/reviews_with_prob.json
